### Creating a class to hold the review and the category

In [9]:
import random

In [5]:
# create a sentiment class to hold the value for each data
class Sentiment:
    Positive = "POSITIVE"
    Negative = "NEGATIVE"
    Neutral = "NEUTRAL"
    
# create another class call reviewClass to hold the review data and the rating
class ReviewClass:
    def __init__(self, review, rate):
        self.review = review
        self.rate = rate
        self.statement = self.get_statement()
    # define another function call get_statement to return the state of each review base on the rated value
    def get_statement(self):
        #set conditional statement to determine each statement per rate
        if (self.rate <= 2):
            return Sentiment.Negative
        elif (self.rate == 3):
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
#write another class that return the review text and the review count and also change the lenght of the positive
class ReviewAdjuster:
    def __init__(self, review):
        self.review = review
    def get_text(self):
        return list(x.review for x in self.review)
    def get_rate(self):
        return list(x.statement for x in self.review)
    def adjuster(self):
        positive = list(filter(lambda x: x.statement == Sentiment.Positive, self.review))
        negative = list(filter(lambda x: x.statement == Sentiment.Negative, self.review))
        positive_shrunk = positive[:len(negative)]
        self.review = negative + positive_shrunk
        random.shuffle(self.review)

    




### import the data from the json file

In [10]:
import json
data_list = list()
with open("Books_small_10000.json") as file:
    for data_file in file:
        data_json = json.loads(data_file)
        review_data = ReviewClass(data_json['reviewText'], data_json['overall'])
        #review = (review_data.review, review_data.rate)
        data_list.append(review_data)



### Preprocessing the data

In [11]:
#calling the train_test_split from sklearn to split the dataset into training and text data 
from sklearn.model_selection import train_test_split
training_data, test_data = train_test_split(data_list, test_size=0.33, random_state=42)
review_adj_train = ReviewAdjuster(training_data)
review_adj_test = ReviewAdjuster(test_data)


In [12]:
review_adj_train.adjuster()
train_x = review_adj_train.get_text()
train_y = review_adj_train.get_rate()




In [13]:
review_adj_test.adjuster()
test_x =review_adj_test.get_text()
test_y = review_adj_test.get_rate()



In [14]:
print("Number of positive in train data:",train_y.count(Sentiment.Positive))
print("Number of Negative in train data:",train_y.count(Sentiment.Negative))
#print("Number of Negative in train data:",train_y.count(Sentiment.Neutral))

Number of positive in train data: 436
Number of Negative in train data: 436


### Bag of word for the preprocessing data

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(train_x)
train_x_vector = vectorizer.transform(train_x)
#print(vectorizer.vocabulary_)
test_x_vector = vectorizer.transform(test_x)

### Model Clasification 

### logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
clf_l = LogisticRegression(random_state=0).fit(train_x_vector, train_y)
print(clf_l.predict(test_x_vector[0]))

['POSITIVE']


### Surport vector machine

In [17]:
from sklearn import svm
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vector, train_y)
clf_svm.predict(test_x_vector[0])

array(['POSITIVE'], dtype='<U8')

### Decision Tree Classifier

In [18]:
from sklearn.tree import DecisionTreeClassifier
clf_gnb = DecisionTreeClassifier(random_state=0)
clf_gnb.fit(train_x_vector, train_y)
y_pred = clf_gnb.predict(test_x_vector[0])

### evaluation of the model

In [19]:
print(clf_l.score(test_x_vector, test_y))
print(clf_svm.score(test_x_vector, test_y))
print(clf_gnb.score(test_x_vector, test_y))

0.8149038461538461
0.7980769230769231
0.6346153846153846


In [20]:
from sklearn.metrics import f1_score
print(f1_score(test_y, clf_svm.predict(test_x_vector), average = None, labels=['POSITIVE', 'NEGATIVE']))
print(f1_score(test_y, clf_l.predict(test_x_vector), average = None, labels=['POSITIVE', 'NEGATIVE']))
print(f1_score(test_y, clf_gnb.predict(test_x_vector), average = None, labels=['POSITIVE', 'NEGATIVE']))


[0.8028169  0.79310345]
[0.82051282 0.808933  ]
[0.63981043 0.62926829]


### Testing the parformance of the model on the new data(unseen data)

In [26]:
test_data = [str(input("Enter your review about our product:"))]
test_data =vectorizer.transform(test_data)
review_feedback = clf_l.predict(test_data)
feedback = str(review_feedback).replace('[', '').replace(']','').replace("'",'')
print("The user review is:",feedback)
if str(feedback) == "POSITIVE":
    print("------------")
    print("Wow!!! thanks for your positive feedback will hope to see you again")
elif str(feedback) == 'NEGATIVE':
    print("------------")
    print("We are sorry,will do you well next time")


Enter your review about our product:Can you please make a tutorial for zoom clone using react native
The user review is: NEGATIVE
------------
We are sorry,will do you well next time


### Saving the model with pickle

In [23]:
import pickle
with open('amazon_sentiment_analysis.pkl', 'wb') as f:
    pickle.dump(clf_l, f)

### Load back the model and to re-text it

In [24]:
with open('amazon_sentiment_analysis.pkl', 'rb') as f:
    data = pickle.load(f)
data.predict(test_x_vector[0])

array(['POSITIVE'], dtype='<U8')

In [ ]:
import jovian

In [ ]:
!pip3 install jovian

In [ ]:
import jovian
jovian.commit()